In [1]:
import json
import csv
import nltk

states = ["ak","al","ar","az","ca","co","ct","de","fl","ga","hi","ia","id","il",
          "in","ks","ky","la","ma","md","me","mi","mn","mo","ms","mt","nc","nd","ne","nh",
          "nj","nm","nv","ny","oh","ok","or","pa","ri","sc","sd","tn","tx","ut","va","vt",
          "wa","wi","wv","wy"]

states_names = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado',
'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois',
'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana',
'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC':'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island',
'SC': 'South Carolina', 'SD': 'South Dakota', 'TN':'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia',
'WA': 'Washington','WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}

afinnfile = open("AFINN-111.txt")
afinnDict = {} # initialize an empty dictionary
#scores_tuples = [line.decode('utf-8').split('\t') for line in afinnline]
#scores = dict((a,int(b)) for a,b in scores_tuples)

for line in afinnfile:
    term, score  = line.split("\t")  # The file is tab-delimited. "\t" means "tab character"
    afinnDict[term] = int(score)  # Convert the score to an integer.

def getState(data):
    if data["place"] != None and data["place"]["country_code"] == "US":
        state = str(data["place"]["full_name"]).lower().split(", ")
        
        if len(state) > 1:
            return state[1]

def isState(state):
    if state in states:
        return True
    return False 

def det_sent(tweet): #helper method for determining net sentiment, will use the sentiment score to appply scores to unscored words.
    terms_count = 0
    score = 0

    for word in tweet:
        stripped_word = word.lower().strip().replace('.','').replace('?','').replace('!','')

        if stripped_word in afinnDict.keys():
            terms_count += 1
            score += afinnDict[stripped_word]
        
    return terms_count, score

def analyze():
    file = "output.txt"
    scored_tweets = []
    tweetsByStates = {}
    
    with open(file, "r") as ins:
        
        for line in ins:          
            
            if len(line) > 1: ## to avoid empty lines 
                tweet = json.loads(line)
           
                if "created_at" in tweet:
                    state = getState(tweet)

                    if isState(state):
                        if "text" in tweet:                           
                            tweetText = tweet["text"]
                            tokenizedTweet = nltk.word_tokenize(tweetText)
                            
                            score = 0
                            numberTerms = 0
                            
                            numberTerms, score = det_sent(tokenizedTweet)
                            
                            if numberTerms > 0:
                                t = [tweetText, state, score, numberTerms]
                                scored_tweets.append(t)
                                
                                if state in tweetsByStates:
                                    tweetsByStates[state].append(score)
                                else:
                                    tweetsByStates[state] = [score]
"""                         
    for [tweetText, state, score, numberTerms] in scored_tweets:
        print("Tweet: " + tweetText)
        print("State: " + state)
        print("Score: " + str(score))
        print("Nº términos analizados: " + str(numberTerms))
        print("----------------------")
"""

    print(tweetsByStates)

In [2]:
analyze()

Tweet: Setting here on break talking to my Sister about the orangutan that flipped me off at the Zoo last Summer lol. It was funny
State: in
Score: 7
Nº términos analizados: 2
----------------------
Tweet: 🎄🎅🎁⛄
Luke Hemmings from @5SOS
who is Santa's 
favorite singer?
Elf-is Presley! 
I love you!❤💚
follow me please?
@Luke5SOS
🎄🎅🎁⛄
17,908
State: mi
Score: 6
Nº términos analizados: 3
----------------------
Tweet: @brittany_vv she won't let me hand mine in lmao
State: ri
Score: 4
Nº términos analizados: 1
----------------------
Tweet: “@trayXO_: College is 30x better than high school. I don't see how people say they miss it.” Fuck high school
State: nj
Score: -4
Nº términos analizados: 3
----------------------
Tweet: @Mondeezzy38 good &amp; nah
State: ca
Score: 3
Nº términos analizados: 1
----------------------
Tweet: Daniels getting butt hurt.
State: tx
Score: -2
Nº términos analizados: 1
----------------------
{'in': [7], 'mi': [6], 'ri': [4], 'nj': [-4], 'ca': [3], 'tx': [-2]}
